In [6]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import os
import warnings
warnings.filterwarnings("ignore")

In [24]:
df_Datos=pd.read_csv('data.csv',encoding= 'unicode_escape')
df_Datos.head(50)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom


In [10]:
df_Datos.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [11]:
df_Datos.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [12]:
df_Datos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


# Cantidad de compras:25900

In [14]:
df_Datos.InvoiceNo.value_counts()

573585     1114
581219      749
581492      731
580729      721
558475      705
           ... 
554023        1
554022        1
554021        1
554020        1
C558901       1
Name: InvoiceNo, Length: 25900, dtype: int64

# Cantidad de clientes: 4372, Total de compras hechas por clientes

In [15]:
df_Datos.CustomerID.value_counts()

17841.0    7983
14911.0    5903
14096.0    5128
12748.0    4642
14606.0    2782
           ... 
15070.0       1
15753.0       1
17065.0       1
16881.0       1
16995.0       1
Name: CustomerID, Length: 4372, dtype: int64

# Cantidad de productos: 4070 ; cantidad total vendida de cada producto(falta multiplicar por la cantidad)

In [19]:
df_Datos.StockCode.value_counts()

85123A    2313
22423     2203
85099B    2159
47566     1727
20725     1639
          ... 
21431        1
22275        1
17001        1
90187A       1
72759        1
Name: StockCode, Length: 4070, dtype: int64

# 50 productos mas vendidos

In [21]:
df_Datos.StockCode.value_counts().head(50)

85123A    2313
22423     2203
85099B    2159
47566     1727
20725     1639
84879     1502
22720     1477
22197     1476
21212     1385
20727     1350
22383     1348
22457     1280
23203     1267
POST      1256
22386     1251
22469     1239
22960     1229
21931     1214
22086     1210
22411     1202
20728     1197
22382     1192
22961     1182
22666     1180
23298     1179
22699     1138
22384     1137
23209     1135
82482     1129
22993     1111
22727     1107
22697     1085
23206     1084
22178     1072
20724     1068
23084     1067
20726     1061
22726     1026
21080     1015
23199     1009
22139      993
22077      986
22138      974
21034      967
22470      963
85099C     960
84946      946
21790      944
82494L     943
23301      937
Name: StockCode, dtype: int64

# Cantidad de compras hechas por pais

In [22]:
df_Datos.Country.value_counts()

United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

In [35]:
df2 = df_Datos[df_Datos['Country'].isin(['United Kingdom'])]
df2

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,año,mes,dia,Precio Total
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010,12,1,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,1,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010,12,1,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,1,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,1,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...
541889,581585,22466,FAIRY TALE COTTAGE NIGHT LIGHT,12,2011-12-09 12:31:00,1.95,15804.0,United Kingdom,2011,12,9,23.40
541890,581586,22061,LARGE CAKE STAND HANGING STRAWBERY,8,2011-12-09 12:49:00,2.95,13113.0,United Kingdom,2011,12,9,23.60
541891,581586,23275,SET OF 3 HANGING OWLS OLLIE BEAK,24,2011-12-09 12:49:00,1.25,13113.0,United Kingdom,2011,12,9,30.00
541892,581586,21217,RED RETROSPOT ROUND CAKE TINS,24,2011-12-09 12:49:00,8.95,13113.0,United Kingdom,2011,12,9,214.80


In [30]:
df_Datos['InvoiceDate'] = pd.to_datetime(df_Datos['InvoiceDate'])

In [31]:
df_Datos['año'] = df_Datos['InvoiceDate'].dt.year
df_Datos['mes'] = df_Datos['InvoiceDate'].dt.month
df_Datos['dia'] = df_Datos['InvoiceDate'].dt.day

In [32]:
df_Datos

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,año,mes,dia
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010,12,1
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010,12,1
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,1
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,1
...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,2011,12,9
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011,12,9
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011,12,9
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011,12,9


In [63]:
Productos_unicos = df_Datos['StockCode'].unique()

In [37]:
df3 = df_Datos[df_Datos['StockCode'].isin(['85123A'])]
df3

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,año,mes,dia,Precio Total
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010,12,1,15.30
49,536373,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,17850.0,United Kingdom,2010,12,1,15.30
66,536375,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,17850.0,United Kingdom,2010,12,1,15.30
220,536390,85123A,WHITE HANGING HEART T-LIGHT HOLDER,64,2010-12-01 10:19:00,2.55,17511.0,United Kingdom,2010,12,1,163.20
262,536394,85123A,WHITE HANGING HEART T-LIGHT HOLDER,32,2010-12-01 10:39:00,2.55,13408.0,United Kingdom,2010,12,1,81.60
...,...,...,...,...,...,...,...,...,...,...,...,...
539342,581439,85123A,CREAM HANGING HEART T-LIGHT HOLDER,1,2011-12-08 16:30:00,5.79,NaN,United Kingdom,2011,12,8,5.79
539979,581452,85123A,WHITE HANGING HEART T-LIGHT HOLDER,32,2011-12-08 18:03:00,2.55,17675.0,United Kingdom,2011,12,8,81.60
540217,581472,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2011-12-08 19:55:00,2.95,15796.0,United Kingdom,2011,12,8,17.70
540837,581492,85123A,CREAM HANGING HEART T-LIGHT HOLDER,3,2011-12-09 10:03:00,5.79,NaN,United Kingdom,2011,12,9,17.37


In [39]:
Total_85123A = df3['Quantity'].sum()
Total_85123A

38830

In [ ]:
for i in range(len(productos_unicos)):
    productos_unicos[i] = df_Datos
    
    
    
    for nombre, grupo in df_Datos.groupby(['StockCode	', 'Quantity']):
    ventas_totales = grupo['ventas'].sum()
    print(f'{nombre}: {ventas_totales}')

In [48]:
Por_mes_85123A = df3.groupby(['mes'])['Quantity'].sum().reset_index().sort_values(by='mes', ascending = True)
Por_mes_85123A


,mes,Quantity
0,1,5522
1,2,1874
2,3,1982
3,4,1907
4,5,3991
5,6,5657
6,7,3002
7,8,2065
8,9,2456
9,10,1687


In [52]:
Orden = [12,1,2,3,4,5,6,7,8,9,10,11]
Por_mes_85123A['Orden'] = Orden
Por_mes_85123A

,mes,Quantity,Orden
0,1,5522,12
1,2,1874,1
2,3,1982,2
3,4,1907,3
4,5,3991,4
5,6,5657,5
6,7,3002,6
7,8,2065,7
8,9,2456,8
9,10,1687,9


In [53]:
PM_Por_mes_85123A = Por_mes_85123A.groupby(['Orden'])['Quantity'].sum().reset_index().sort_values(by='Orden', ascending = True)
PM_Por_mes_85123A

,Orden,Quantity
0,1,1874
1,2,1982
2,3,1907
3,4,3991
4,5,5657
5,6,3002
6,7,2065
7,8,2456
8,9,1687
9,10,4648


In [56]:
rolling_mean1 = PM_Por_mes_85123A.rolling(window=3).mean()
print(rolling_mean1)

    Orden     Quantity
0     NaN          NaN
1     NaN          NaN
2     2.0  1921.000000
3     3.0  2626.666667
4     4.0  3851.666667
5     5.0  4216.666667
6     6.0  3574.666667
7     7.0  2507.666667
8     8.0  2069.333333
9     9.0  2930.333333
10   10.0  3458.000000
11   11.0  4736.333333


In [58]:
rolling_mean = Por_mes_85123A.rolling(window=3).mean()
print(rolling_mean)

     mes     Quantity  Orden
0    NaN          NaN    NaN
1    NaN          NaN    NaN
2    2.0  3126.000000    5.0
3    3.0  1921.000000    2.0
4    4.0  2626.666667    3.0
5    5.0  3851.666667    4.0
6    6.0  4216.666667    5.0
7    7.0  3574.666667    6.0
8    8.0  2507.666667    7.0
9    9.0  2069.333333    8.0
10  10.0  2930.333333    9.0
11  11.0  3458.000000   10.0


In [38]:
Productos = df_Datos.groupby(['StockCode'])['Quantity'].count().reset_index().sort_values(by='Quantity', ascending = False)
Productos.head(10)

,StockCode,Quantity
3536,85123A,2313
1348,22423,2203
3515,85099B,2159
2733,47566,1727
180,20725,1639
3305,84879,1502
1631,22720,1477
1147,22197,1476
454,21212,1385
182,20727,1350


In [33]:
df_Datos['Precio Total'] = df_Datos['Quantity']*df_Datos['UnitPrice']

In [34]:
df_Datos

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,año,mes,dia,Precio Total
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010,12,1,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,1,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010,12,1,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,1,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,1,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,2011,12,9,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011,12,9,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011,12,9,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011,12,9,16.60
